<a href="https://colab.research.google.com/github/csrviswa/Sentiment_Analysis/blob/master/Neighborhoods_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pip    
!pip install BeautifulSoup4
!pip install lxml
!pip install requests